**Generative Adversarial Networks (GANs)** are a class of machine learning frameworks designed to generate new, synthetic data samples that resemble a given training dataset. GANs consist of two neural networks: the Generator and the Discriminator. The Generator creates fake data, while the Discriminator evaluates the authenticity of the data (whether it's real or fake). The two networks are trained together in a process where the Generator tries to improve its ability to create realistic data, and the Discriminator tries to get better at identifying fake data. This adversarial process continues until the Generator produces data that the Discriminator cannot easily distinguish from real data.

**Use Cases for GANs**
GANs are widely used in various applications, including:

- Image Generation: Creating realistic images from noise, such as generating faces that don't exist or producing art in the style of famous painters.
Data Augmentation: Generating synthetic data to augment training datasets, which can be particularly useful in fields like medical imaging where data may be scarce.  
- Style Transfer: Applying the style of one image (e.g., a painting) to the content of another (e.g., a photograph).  
- Image-to-Image Translation: Transforming images from one domain to another, such as converting sketches into photos or daytime scenes into nighttime scenes.
- Text-to-Image Synthesis: Generating images from textual descriptions, enabling the creation of visuals based solely on text input.  

**Principal Elements of GANs**

- Generator:  
The Generator's job is to create synthetic data samples that mimic the real data.  
Typically a neural network that takes random noise as input and transforms it into a structured output, like an image or audio signal.
- Discriminator:  
The Discriminator's job is to distinguish between real data from the training set and fake data produced by the Generator.  
Usually a binary classifier that outputs a probability indicating whether a given input is real or fake.  
- Adversarial Training:  
Both the Generator and Discriminator are trained simultaneously in a competitive process. The Generator aims to fool the Discriminator, while the Discriminator aims to correctly identify fake data.  

- Loss Functions:  
The Generator tries to minimize the Discriminator's ability to detect fake data, while the Discriminator tries to maximize its accuracy.
Latent Space:  
The input to the Generator is typically random noise drawn from a latent space (e.g., a Gaussian distribution). This noise is transformed by the Generator into a meaningful output.  
- Training Process:  
The training involves alternating between updating the Discriminator and the Generator. Over time, the Generator learns to produce more realistic data, while the Discriminator becomes more adept at distinguishing real from fake until the Generator's outputs become indistinguishable from real data

**PROBLEM DESCRIPTION**  
The problem we address is image generation, specifically using the CelebA dataset to generate realistic images of faces. The task is to train a GAN to produce new, realistic-looking images of human faces that could resemble the real images in the dataset.

https://drive.google.com/drive/folders/0B7EVK8r0v71pTUZsaXdaSnZBZzg?resourcekey=0-rJlzl934LzC-Xp28GeIBzQ

In [28]:
ds_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\egzlz\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [29]:
(ds_train, ds_test), ds_info = tfds.load(
    'celeb_a',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Size...: 0 MiB [00:00, ? MiB/s]1/4 [00:00<00:02,  1.29 url/s]
Dl Completed...:  25%|██▌       | 1/4 [00:00<00:02,  1.24 url/s]


NonMatchingChecksumError: Artifact https://drive.google.com/uc?export=download&id=0B7EVK8r0v71pZjFTYXZWM3FlRnM, downloaded to C:\Users\egzlz\tensorflow_datasets\downloads\ucexport_download_id_0B7EVK8r0v71pZjFTYXZWM3FlDDaXUAQO8EGH_a7VqGNLRtW52mva1LzDrb-V723OQN8.tmp.85a2524575c44ea0af78619152b427dc\download, has wrong checksum:
* Expected: UrlInfo(size=1.34 GiB, checksum='46fb89443c578308acf364d7d379fe1b9efb793042c0af734b6112e4fd3a8c74', filename='img_align_celeba.zip')
* Got: UrlInfo(size=2.37 KiB, checksum='411fbe113ae59426d9d8de8ad5fe9f0a047161e03af00e7fbdaefd07f2ad49d1', filename='download')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# Definición del generador
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

# Definición del discriminador
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

# Instanciación del generador y discriminador
generator = build_generator()
discriminator = build_discriminator()

# Compilación del discriminador
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

# Definición de la GAN
discriminator.trainable = False
gan_input = tf.keras.Input(shape=(100,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = tf.keras.Model(gan_input, gan_output)

# Compilación de la GAN
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Entrenamiento de la GAN
import numpy as np

# Función de entrenamiento de la GAN
def train_gan(gan, generator, discriminator, epochs, batch_size):
    for epoch in range(epochs):
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)
        
        real_images = ... # Cargar un batch de imágenes reales aquí
        
        labels_real = np.ones((batch_size, 1))
        labels_fake = np.zeros((batch_size, 1))
        
        # Entrenar el discriminador
        d_loss_real = discriminator.train_on_batch(real_images, labels_real)
        d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)
        
        # Entrenar la GAN (actualizando el generador)
        noise = np.random.normal(0, 1, (batch_size, 100))
        gan_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
        
        print(f"Epoch {epoch + 1}/{epochs} - D loss: {d_loss_real + d_loss_fake}, G loss: {gan_loss}")

# Entrenamiento (se necesita un dataset de imágenes reales para las GANs)
train_gan(gan, generator, discriminator, epochs=10000, batch_size=128)

4/4 [==============================] - 0s 66ms/step


AttributeError: 'ellipsis' object has no attribute 'shape'